<a href="https://colab.research.google.com/github/piusAI/AllAboutDL/blob/main/section04_01_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values


<class 'pandas.core.series.Series'>


.iloc :  Extract entire data frame

In [ ]:
print(type(dataset.iloc[:,3:-1]))

<class 'pandas.core.frame.DataFrame'>


.values를 잡아주지 않으면 그냥 Pandas.dataFrame이 나옴

Pandas DataFrame : row-column으로의 table Data

In [ ]:
print("Feature:\n", X)

print("\n Leave Parm: \n", y)

Feature:
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]

 Leave Parm: 
 [1 0 1 ... 1 1 0]


Label = Leave Param을 예측하기 위한 Classification

고객이 빠져나가는지 확인하기 위해!

### Encoding categorical data

Label Encoding the "Gender" column

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2]) # Exchange Female:0, Male:1

LabelEncoder

Male, Female과 같은 string=> int

| 메서드               | 하는 일                                                            | 예시                              |
| ----------------- | --------------------------------------------------------------- | ------------------------------- |
| `fit()`           | 데이터에서 변환 기준을 **학습**  | `['Male', 'Female']` → 매핑 기억만 함 |
| `transform()`     | **이미 학습한 기준으로 변환만 수행**                                          | `['Male', 'Female']` → `[1, 0]` |
| `fit_transform()` | 위 두 과정을 **한 번에 처리** (학습 + 변환)                                   | `['Male', 'Female']` → `[1, 0]` |

-> fit은 그냥 string -> int이상의 의미를 가짐.

알아서 어떤 기준을 갖고 기계가 스스로 결정!

In [21]:
print(X[:,2])

[0 0 0 ... 0 1 0]


In [16]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X = np.array(ct.fit_transform(X))


[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


In [31]:
print(ct.fit_transform(X)) #보통 sparese matrix로 나오기도해서

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


<Preprocess, Template 활용해서>

Geopgraphy, 지역을 one-hot code로 수정함

In [29]:
print(X)

[[1.0 0.0 1.0 ... 1 1 101348.88]
 [1.0 0.0 0.0 ... 0 1 112542.58]
 [1.0 0.0 1.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 1.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 1.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

### Feature Scaling

In [33]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Standard Scaler!

- 특성 1: 나이 = [20, 30, 40]
- 특성 2: 연봉 = [30,000,000, 60,000,000, 100,000,000]

이렇게 각각의 Feature Scale의 차이를 없애주는것


```
# X_scaled = (X - 평균) / 표준편차

```



| 특성 | 평균  | 표준편차 | 스케일링 결과              |
| -- | --- | ---- | -------------------- |
| 나이 | 30  | 10   | \[-1, 0, 1]          |
| 연봉 | 63M | 29M  | \[-1.14, -0.1, 1.24] |


-> Train data에만 standardscaler해주기!

## Part 2 - Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()
# integrated Keras, into tf

### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation='relu'))

### Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units =1, activation = 'sigmoid'))

=> Regression하면 최종적으로 연속적인 실수 예측해야해서 Activation 함수 안했다

## Part 3 - Training the ANN

### Compiling the ANN

Optimizer?

In [ ]:
ann.compile(optimizer='adam', loss = "mean_squared_error")

adam algorithm으로 MSE 로 loss function잡음

### Training the ANN on the Training set

In [ ]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

In [ ]:
y_pred = ann.predict(x_test, batch_size=32)
np.set_printoptions(precision=2) # 소수점 두개만 parameter로
print( np.concatenate(( y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)),1 ) )
 # concatenate를 활용해서
 # [Predict - Label] 확인

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

### Making the Confusion Matrix